# Quantum Dynamics

In this project, we use the Crank-Nicolson method, an implicit and numerically stable finite difference method, to numerically solve the time-dependent Schrödinger equation in one and two dimensions. To this end, we have simulated a Gaussian wave packet incident on a finite potential well (in one dimension) and a plane wave incident on a double slit (in two dimensions). We will first describe how we obtained the solution to the Schrödinger equation in 1D, followed by an animation of the evolution of the wave packet in time, after which we do the same for the two dimensional case. We conclude with a plot showing the interference at the location of an imaginary screen.

In [1]:
# import the different Python packages used

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse import spdiags
from scipy.sparse.linalg import bicgstab
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
import sys

##  Crank-Nicolson method in 1D

the Crank-Nicolson method in 1D,


$$ i \hbar \frac{\psi(x,t + \Delta t) - \psi(x,t)}{\Delta t} = \frac{\hat{H} \psi(x,t) + \hat{H} \psi(x,t + \Delta t)}{2} $$


can be rewritten as


$$ \left(\frac{i \hbar}{\Delta t} - \frac{1}{2} \hat{H}(t + \Delta t)\right) \psi(x,t+\Delta t) = \left(\frac{i \hbar}{\Delta t} + \frac{1}{2} \hat{H}(t)\right) \psi(x,t) $$


The time-dependent Schrödinger equation in one dimension is equal to:


$$ \hat{H}(t) \psi(x,t) = \left(- \frac{ \hbar^2}{2m} \frac{\partial^2}{\partial x^2} + V(x,t)\right) \psi(x,t) $$


and we can use the $2^{nd}$ order central difference equation to approximate the differential operator:


$$ \frac{\partial^2}{\partial x^2} \psi(x,t) = \frac{\psi(x + \Delta x,t) -2\psi(x,t) + \psi(x - \Delta x)}{(\Delta x)^2} $$


resulting in


$$ \hat{H}(t) \psi(x,t) = - \frac{ \hbar^2}{2m} \left(\frac{\psi(x + \Delta x,t) -2\psi(x,t) + \psi(x - \Delta x,t)}{(\Delta x)^2}\right) + V(x,t)\psi(x,t) $$


If we use this equation in the Crank-Nicolson method, we obtain the following result:


$$ \frac{i\hbar}{\Delta t} \psi(x,t+\Delta t) -\frac{1}{2} \left(- \frac{ \hbar^2}{2m} \left(\frac{\psi(x + \Delta x,t + \Delta t) -2\psi(x,t + \Delta t) + \psi(x - \Delta x, t + \Delta t)}{(\Delta x)^2}\right) + V(x, t + \Delta t) \psi(x, t + \Delta t)\right) = \frac{i\hbar}{\Delta t} \psi(x,t) +\frac{1}{2} \left(- \frac{ \hbar^2}{2m} \left(\frac{\psi(x + \Delta x,t) -2\psi(x,t) + \psi(x - \Delta x, t)}{(\Delta x)^2}\right) + V(x, t) \psi(x, t)\right)$$


which we can rewrite to


$$ \frac{\hbar^2}{4m(\Delta x)^2} \psi(x +\Delta x, t+\Delta t) + \left(\frac{i \hbar}{\Delta t} - \frac{\hbar^2}{2m(\Delta x)^2} - \frac{1}{2} V(x,t + \Delta t)\right) \psi(x,t+\Delta t) + \frac{\hbar^2}{4m(\Delta x)^2} \psi(x -\Delta x, t+\Delta t) = -\frac{\hbar^2}{4m(\Delta x)^2} \psi(x +\Delta x, t) + \left(\frac{i \hbar}{\Delta t} + \frac{\hbar^2}{2m(\Delta x)^2} + \frac{1}{2} V(x,t)\right) \psi(x,t) -\frac{\hbar^2}{4m(\Delta x)^2} \psi(x -\Delta x, t) $$


To obtain a numerical solution we use the biconjugate gradient stabilized method, which solves the linear system $Ax = b$, so we have to rewrite the equation in the following format:


$$ A\overrightarrow{\psi}(t + \Delta t) = B\overrightarrow{\psi}(t) $$


When we set $\hbar = m$ equal to 1, and name the following constants $a$ and $b$:


$$ a = \frac{i}{\Delta t}\quad, \quad b = \frac{1}{4(\Delta x)^2} $$


we can construct both tridiagonal matrices and the vector which will be used to calculate the evolution of the wave function in time:

$$ A = \begin{pmatrix}
a-2b-\frac{1}{2}V & b & 0 \\
b & a-2b-\frac{1}{2}V & b \\
0 & b & a-2b-\frac{1}{2}V \\ 
0 & 0 & b & \ddots \\ 
\end{pmatrix} $$

$$ B = \begin{pmatrix}
a+2b+\frac{1}{2}V & -b & 0 \\
-b & a+2b+\frac{1}{2}V & -b \\
0 & -b & a+2b+\frac{1}{2}V \\ 
0 & 0 & -b & \ddots \\ 
\end{pmatrix} $$

$$ \overrightarrow{\psi}(t) = \begin{pmatrix}
\psi(x=x_0, t)\\
\psi(x_0 + \Delta x,t)\\
\psi(x_0 + 2\Delta x,t)\\
\psi(x_0 + 3\Delta x,t)\\
\vdots\\
\end{pmatrix} $$


## Gaussian wave packet incident on a finite potential well

In this part of the project, we simulate a Gaussian wave packet incident on a finite potential well, making use of the Crank-Nicolson method and the biconjugate gradient stabilized method described above. This is visualised using a 2D animation.

In [2]:
# initialise the parameters used for the 1D case

def initialise_1D():
    global N, L, x, dx, dt, x0, sig, k, V0, a
    N = 1001  # number of gridpoints on the x-asis
    L = 100.   # space length of the simulation
    x = np.linspace(0, L, N)  
    dx = x[1]-x[0]  # grid spacing
    dt = 0.05  # time step
    x0 = 10.  # initial position of the center of the wave packet
    sig = 3.  # wave packet width
    k = 3.  # wave number
    V0 = 5 # potential height
    a = 10.  # width of the barrier
    

In [3]:
# construct a Gaussian wave packet with its center at x0, momentum k and width sig
def Gaussian_wave_packet(sig, x, x0, k,dx):
    psi = np.exp(-(x-x0)**2/(2*sig**2)) * np.exp(1.j*k*x)
    psi /= np.sqrt(sum(abs(psi)**2*dx)) 
    return psi

# construct a potential well
def potential_well(V0, x, L, a):
    V = V0*(abs(x-L/2.) < a/2)
    return V


In [4]:
# Calculate the tridiagonal matrices A and B used in the BiCGSTAB method
def calculate_AB_1D(dt, dx, V):
    
    # define the constants a and b
    a = 1.j/dt
    b = 1/(4*dx**2)
    
    # construct the sparse matrices A and B
    A = diags([b, a-2*b-0.5*V, b], [-1, 0, 1], shape=(N, N))
    B = diags([-b, a+2*b+0.5*V, -b], [-1, 0, 1], shape=(N, N))
    return A, B

# calculate the wave function for the next time step using the BiCGSTAB method
def propagate_1D():
    global psi
    psi = bicgstab(A, B*psi, x0 = psi)[0]

In [5]:
# define the functions used for the animation

# creating the base frame for the animation
def init_1D():
    psi_line.set_data([],[])
    V_line.set_data([],[])
    return psi_line, V_line,

# construct the animation function
def animate_1D(f):
    propagate_1D()
    psi_line.set_data(x, abs(psi))
    V_line.set_data(x, V)
    return psi_line, V_line,
    

In [6]:
# running the animation

initialise_1D()
fig = plt.figure()
ax = plt.axes(xlim=(0, 100), ylim=(-0.5, 0.5))
psi_line, = ax.plot([],[])
V_line, = ax.plot([],[])  
V = np.vectorize(potential_well)(V0, x, L, a)  # vectorise the potential so that it can be used to construct the sparse matrices
A, B = calculate_AB_1D(dt, dx, V)
psi = Gaussian_wave_packet(sig, x, x0, k, dx)

ax.set_title("Propagation of a Gaussian wave packet incident on a finite potential well")
ax.set_xlabel("$x$")
ax.set_ylabel("$\psi(x)$")

anim = animation.FuncAnimation(fig, animate_1D, frames=2000, interval=20, blit=True, init_func=init_1D, repeat=False)

plt.show()


In [10]:
# Define the functions for running the animation for the transmission

# Energy of the wave
E = np.linspace(0.5, 5.0, 20)

# Array with zeros to fill up with values for the transmission
T = np.zeros(len(E))

# Creating the base frame for the animation
def init_ani_trans(psi, x, V, V0):
    fig, ax = plt.subplots()
    plt.plot(x, 1.5*max(abs(psi)**2)*V/V0, 'r')
    ims = []  # list for storing all of the created artists

    return fig, ax, ims

# Design to plot the transmission
def plot_transmission(E, T):
    plt.figure()
    plt.title("Transmission of a gaussian wave packet through a potential barrier")
    plt.xlabel('Energy')
    plt.ylabel('Transmission')
    plt.axis((0, np.max(E), 0, 1.1))
    plt.plot(E, T)

    plt.show()

In [11]:
# The function for the Transmission

# Creating a function to calculate the transmission coefficient
def CrankNicolson(A, B, x, psi, a, L, dx, ims=None):
    
    y = 2*np.max(abs(psi))**2       #max heigth
    C = CN = i = 0                  #start value
    while CN>=C or C<10e-6:         #norm in the barrier
        C = CN
        if ims!=None and i%4==0:
            plt.axis((0,L,0,y))
            im, = plt.plot(x, np.abs(psi)**2, 'b')
            ims.append([im])  # add the image to the list of artists
            
        [psi, error] = bicgstab(A, B*psi, x0=psi)
        if error != 0: sys.exit("Bicgstab did not converge") # error if the bicgstab function does not converge
            
        i = i+1
        
        CN = sum(abs(psi[int(round((L - a)/(2*dx))):int(round((L + a)/(2*dx)))])**2)*dx # New norm
    
    return psi, sum(abs(psi[int(round((L - a)/(2*dx))):])**2)*dx / (sum(abs(psi)**2)*dx)

# Loop to simulate the Gaussian wave for different k-values and thus different transmission coefficients
for i,k in enumerate(np.sqrt(2*E*V0)):
    
    psi = Gaussian_wave_packet(sig, x, (L-a)/2. - 3*sig, k, dx)
    
    A,B = calculate_AB_1D(dt, dx, V)
    
    if False:
        fig, ax, ims = init_ani_transmission(psi, x, V, V0)
        psi, T[i] = CrankNicolson(A, B, x, psi, a, L, dx, ims)
        

        im_ani = animation.ArtistAnimation(fig, ims, interval=10, repeat_delay=500, blit=True)
        plt.show
    else:
        psi, T[i] = CrankNicolson(A, B, x, psi, a, L, dx)
    

if True: plot_transmission(E, T)

## Crank-Nicolson method in 2D

The Crank-Nicolson method in 2D is


$$ \left(\frac{i \hbar}{\Delta t} - \frac{1}{2}\hat{H}(t+\Delta t)\right) \psi(x,y,t+\Delta t) = \left(\frac{i \hbar}{\Delta t} + \frac{1}{2}\hat{H}(t)\right) \psi(x,y,t) $$


and the time-dependent Schrödinger equation in 2D combined with the $2^{nd}$ order central difference equation leads to the following equation:


$$ \hat{H}(t) \psi(x,y,t) = - \frac{ \hbar^2}{2m} \left(\frac{\psi(x + \Delta x,y,t) -2\psi(x,y,t) + \psi(x - \Delta x,y,t)}{(\Delta x)^2} + \frac{\psi(x,y+\Delta y,t) -2\psi(x,y,t) + \psi(x,y-\Delta y,t)}{(\Delta y)^2}\right) + V(x,y,t)\psi(x,y,t) $$


If we combine the two and use again the same constants $a$ and $b$ as in the 1D method, and use the fact that we choose the same gridspacing for x and y, we end up with the following:


$$ b\psi(x+\Delta x,y,t+\Delta t) + \left(a-4b-\frac{1}{2}V(x,y,t+\Delta t)\right)\psi(x,y,t+\Delta t) + b\psi(x-\Delta x,y,t+\Delta t) + b\psi(x,y+\Delta y,t+\Delta t) + b\psi(x,y-\Delta y,t+\Delta t) = -b\psi(x+\Delta x,y,t) + \left(a+4b+\frac{1}{2}V(x,y,t)\right)\psi(x,y,t) -b\psi(x-\Delta x,y,t) - b\psi(x,y+\Delta y, t) - b\psi(x,y - \Delta y,t) $$


which we again have to rewrite in the format


$$ A\overrightarrow{\psi}(t + \Delta t) = B\overrightarrow{\psi}(t) $$


We now construct both matrices and the vector which will be used to calculate the evolution of the wave function in time. both outer most diagonals are situated at a distance N from the main diagonal, and to prevent the inclusion of coordinates (x,y) outside of the interval, the first of every N values in the off-diagonals is set to zero.


$$ A = \begin{pmatrix}
\ddots & b & 0 & 0 & 0 & \cdots & b & a-4b-\frac{1}{2}V & b & 0 & \cdots & b & 0 & 0 & 0 & \ddots \\
\ddots & 0 & b & 0 & 0 & \cdots & 0 & b & a-2b-\frac{1}{2}V & b & \cdots & 0 & b & 0 & 0 & \ddots \\
\ddots & 0 & 0 & b & 0 & \cdots & 0 & 0 & b & a-2b-\frac{1}{2}V & \cdots & 0 & 0 & b & 0 & \ddots \\ 
\ddots & 0 & 0 & 0 & b & \cdots & 0 & 0 & 0 & b & \cdots & 0 & 0 & 0 & b & \ddots\\ 
\end{pmatrix} $$


$$ B = \begin{pmatrix}
\ddots & -b & 0 & 0 & 0 & \cdots & -b & a+4b+\frac{1}{2}V & -b & 0 & \cdots & -b & 0 & 0 & 0 & \ddots \\
\ddots & 0 & -b & 0 & 0 & \cdots & 0 & -b & a+2b+\frac{1}{2}V & -b & \cdots & 0 & -b & 0 & 0 & \ddots \\
\ddots & 0 & 0 & -b & 0 & \cdots & 0 & 0 & -b & a+2b+\frac{1}{2}V & \cdots & 0 & 0 & -b & 0 & \ddots \\ 
\ddots & 0 & 0 & 0 & -b & \cdots & 0 & 0 & 0 & -b & \cdots & 0 & 0 & 0 & -b & \ddots\\ 
\end{pmatrix} $$


$$ \overrightarrow{\psi}(t) = \begin{pmatrix}
\psi(x=x_0,y=y_0,t)\\
\psi(x_0 + \Delta x,y_0,t)\\
\psi(x_0 + 2\Delta x,y_0,t)\\
\psi(x_0 + 3\Delta x,y_0,t)\\
\vdots\\
\psi(x_0 + N\Delta x,y_0,t)\\
\psi(x_0,y_0 + \Delta y,t)\\
\psi(x_0 +\Delta x, y_0 + \Delta y,t)\\
\vdots\\
\end{pmatrix} $$



## Plane wave incident on a double slit

In this part of the project, we simulate a plane wave incident on a double slit, making use of the Crank-Nicolson method and the biconjugate gradient stabilized method described above. The wave propagation is visualised using a 3D animation, and the interference pattern on an imaginary screen is plotted.

In [89]:
# initialise the parameters used for the 2D case

def initialise_2D():
    global N, x, y, dx, dt, k, V, slx, sly, d, xpos, ypos, steps, interference
    N = 100  # number of gridpoints on each axis
    x = np.linspace(0, 100, N)
    dx = x[1]-x[0]  # spacing between two grid points
    y = x
    x, y = np.meshgrid(x, y)
    dt = 0.05  # time step
    k = 5  # wave number

    V = np.zeros((N, N))  # matrix to hold the potential values
    slx = 2  # horizontal slit length
    sly = 4  # vertical slit length
    d = 20  # slit separation distance
    xpos = 40  # x-location of the potential barrier
    ypos = 36  # y-location of the upper slit
    
    steps = 2000  # number of times wave function is evaluated in time in order to plot the interference pattern
    interference = np.zeros((N,steps))  # matrix to hold all ... at the location of the imaginary screen at each time step
    

In [90]:
# construct a plane wave with momentum k
def plane_wave(x, k):
    psi = np.exp(-x*(1 + 1.j*k))
    return psi

# construct a double slit potential with high barriers to ensure total reflection
def double_slit_potential(V, slx, sly, d):
    V[0:(ypos//dx),(xpos//dx):((xpos+slx)//dx)] = 1e5
    V[((ypos+sly)//dx):((ypos+sly+d)//dx),(xpos//dx):((xpos+slx)//dx)] = 1e5
    V[((ypos+2*sly+d)//dx):(N+1),(xpos//dx):((xpos+slx)//dx)] = 1e5
    return V   
    

In [91]:
# calculate the matrices A and B used in the BiCGSTAB method
# we have to use a different method for constructing the sparse matrices because not all values within each diagonal are the same
# We first construct arrays with zeros only, and then fill them with the correct values

def calculate_AB_2D(dt, dx, V):
    
    # define the constants a and b
    a = 1.j/dt
    b = 1/(4*dx**2)
    
    A_diag = np.zeros(N**2, dtype=complex)
    A_off_diag = np.zeros(N**2, dtype=complex)
    B_diag = np.zeros(N**2, dtype=complex)
    B_off_diag = np.zeros(N**2, dtype=complex)
    
    A_diag.fill(a-4*b)
    A_diag -= 0.5*V
    A_off_diag.fill(b)
    B_diag.fill(a+4*b)
    B_diag += 0.5*V
    B_off_diag.fill(-b)
    A_off_diag[::N]=0; B_off_diag[::N]=0  # ensures that the first of each N values within the string are zero
    
    # construct the sparse matrices A and B
    A = spdiags([A_off_diag, A_off_diag, A_diag, A_off_diag, A_off_diag], [-N, -1, 0, 1, N], N**2, N**2)
    B = spdiags([B_off_diag, B_off_diag, B_diag, B_off_diag, B_off_diag], [-N, -1, 0, 1, N], N**2, N**2)
    return A, B
              
# calculate the wave function for the next time step using the BiCGSTAB method
def propagate_2D():
    global psi
    psi = bicgstab(A, B*psi, x0 = psi)[0]


In [92]:
# define the functions used for the animation

# creating the base frame for the animation
def init_2D():
    im.set_data([[]])
    return im

# construct the animation function
def animate_2D(f):
    propagate_2D()
    im.set_data(abs(np.reshape(psi, (N,N))))
    return im
    

In [93]:
# running the animation and plotting the interference pattern

initialise_2D()
V = double_slit_potential(V, slx, sly, d)
V = np.reshape(V, -1)  # reshape the matrix into an array to be used for calculating the sparse matrices
A, B = calculate_AB_2D(dt, dx, V)
psi = plane_wave(x, k)
psi = np.reshape(psi, -1)  # reshape the matrix into an array to be used in the BiCGSTAB method

fig = plt.figure()
ax = plt.axes(xlim=(0, 100), ylim=(100, 0))
im = ax.imshow(abs(np.reshape(psi, (N,N))), cmap="jet", vmin=0, vmax=abs(psi).max())

ax.set_title("Propagation of a plane wave incident on a double slit")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")

anim = animation.FuncAnimation(fig, animate_2D, frames=2300, init_func=init_2D, interval=20, repeat=False)
                              
plt.show()

# interference pattern

psi = plane_wave(x, k)
interference[:,0] = abs(psi)[:,75]  # saving the values of the wave function at the location of the screen
psi = np.reshape(psi, -1)

for i in range(1,steps):
    propagate_2D()
    interference[:,i] = abs(np.reshape(psi, (N,N)))[:,75]

plt.plot(y[:,0], np.sum(interference, axis=1))  # summing to obtain the pattern
plt.xlabel('y')
plt.ylabel('$ \sum_{i=1}^{%g} |\psi(75,y,i\Delta t)| $'%(steps))
plt.title('Accumulated interference pattern')
plt.show()


C:\Users\Stefani\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Stefani\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Stefani\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
